In [2]:
from PIL import Image

#カレントディレクトリの画像指定
img_dir = 'images/o365_images/厚労省.png'

#画像オブジェクト作成
img = Image.open(img_dir)

#オリジナル画像の幅・高さ確認
width, height = img.size
print("オリジナル画像の"+"幅: "+str(width),"高さ:"+str(height))

#リサイズ
w_shrink = 4  #縮小幅の割合指定
h_shrink = 4  #縮小高さの割合指定
img_resize = img.resize((int(width/w_shrink),int(height/h_shrink)))

#リサイズした画像の幅・高さ確認
width, height = img_resize.size
print("リサイズした画像の"+"幅: "+str(width),"高さ:"+str(height))

#リサイズした画像を上書き保存
img_resize.save(img_dir)

オリジナル画像の幅: 1898 高さ:818
リサイズした画像の幅: 474 高さ:204


In [3]:
import openpyxl

#Excelファイル名
name = "画像貼り付け.xlsx"

#Excelファイル作成
workbook = openpyxl.Workbook(name)

#保存
workbook.save(name)

In [4]:
#Excelファイル読込
workbook = openpyxl.load_workbook(name)
sheet = workbook.active

#画像を選択
img_to_excel = openpyxl.drawing.image.Image(img_dir)

#指定の位置に画像を添付
sheet.add_image(img_to_excel, 'B3')

#保存
workbook.save(name)